# Improving your model
Here, you'll improve on your benchmark model using pipelines. Because the budget consists of both text and numeric data, you'll learn to how build pipielines that process multiple types of data. You'll also explore how the flexibility of the pipeline workflow makes testing different approaches efficient, even in complicated problems like this one!
# 1. Pipelines, feature & text preprocessing
## 1.1 Instantiate pipeline
In order to make your life easier as you start to work with all of the data in your original DataFrame, `df`, it's time to turn to one of scikit-learn's most useful objects: the `Pipeline`.

For the next few exercises, you'll reacquaint yourself with pipelines and train a classifier on some synthetic (sample) data of multiple datatypes before using the same techniques on the main dataset.

The sample data is stored in the DataFrame, `sample_df`, which has three kinds of feature data: numeric, text, and numeric with missing values. It also has a label column with two classes, `a` and `b`.

In this exercise, your job is to instantiate a pipeline that trains using the `numeric` column of the sample data.

### Instructions:
* Import `Pipeline` from `sklearn.pipeline`.
* Create training and test sets using the numeric data only. Do this by specifying `sample_df[['numeric']]` in `train_test_split()`.
* Instantiate a pipeline as `pl` by adding the classifier step. Use a name of `'clf'` and the same classifier from Chapter 2: `OneVsRestClassifier(LogisticRegression())`.
* Fit your pipeline to the training data and compute its accuracy to see it in action! Since this is toy data, you'll use the default scoring method for now. In the next chapter, you'll return to log loss scoring.

In [1]:
import numpy as np
import pandas as pd

rng = np.random.RandomState(123)

SIZE = 1000

sample_data = {
    'numeric': rng.normal(0, 10, size=SIZE),
    'text': rng.choice(['', 'foo', 'bar', 'foo bar', 'bar foo'], size=SIZE),
    'with_missing': rng.normal(loc=3, size=SIZE)
}

sample_df = pd.DataFrame(sample_data)

sample_df.loc[rng.choice(sample_df.index, size=np.floor_divide(sample_df.shape[0], 5)), 'with_missing'] = np.nan

foo_values = sample_df.text.str.contains('foo') * 10
bar_values = sample_df.text.str.contains('bar') * -25
no_text = ((foo_values + bar_values) == 0) * 1

val = 2 * sample_df.numeric + -2 * (foo_values + bar_values + no_text) + 4 * sample_df.with_missing.fillna(3)
val += rng.normal(0, 8, size=SIZE)

sample_df['label'] = np.where(val > np.median(val), 'a', 'b')

print(sample_df.head())

     numeric     text  with_missing label
0 -10.856306               4.433240     b
1   9.973454      foo      4.310229     b
2   2.829785  foo bar      2.469828     a
3 -15.062947               2.852981     b
4  -5.786003  foo bar      1.826475     a


In [2]:
# Import Pipeline
from sklearn.pipeline import Pipeline

# Import other necessary modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Split and select numeric data only, no nans 
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=22)

# Instantiate Pipeline object: pl
pl = Pipeline([
        ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
    ])

# Fit the pipeline to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - numeric, no nans: ", accuracy)


Accuracy on sample data - numeric, no nans:  0.62


Now it's time to incorporate numeric data with missing values by adding a preprocessing step!

## 1.2 Preprocessing numeric features
What would have happened if you had included the with `'with_missing'` column in the last exercise? Without imputing missing values, the pipeline would __not__ be happy (try it and see). So, in this exercise you'll improve your pipeline a bit by using the `Imputer()` imputation transformer from scikit-learn to fill in missing values in your sample data.

By default, the [imputer transformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) replaces NaNs with the mean value of the column. That's a good enough imputation strategy for the sample data, so you won't need to pass anything extra to the imputer.

After importing the transformer, you will edit the steps list used in the previous exercise by inserting a `(name, transform)` tuple. Recall that steps are processed sequentially, so make sure the new tuple encoding your _preprocessing_ step is put in the right place.

The `sample_df` is in the workspace, in case you'd like to take another look. Make sure to select __both__ numeric columns- in the previous exercise we couldn't use `with_missing` because we had no preprocessing step!

### Instructions:
* Import `Imputer` from `sklearn.preprocessing`.
* Create training and test sets by selecting the correct subset of `sample_df`: `'numeric'` and `'with_missing'`.
* Add the tuple `('imp', Imputer())` to the correct position in the pipeline. `Pipeline` processes steps sequentially, so the imputation step should come _before_ the classifier step.
* Complete the `.fit()` and `.score()` methods to fit the pipeline to the data and compute the accuracy.

In [3]:
# Import the Imputer object
from sklearn.impute import SimpleImputer

# Create training and test sets using only numeric data
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=456)

# Insantiate Pipeline object: pl
pl = Pipeline([
        ('imp', SimpleImputer()),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
    ])

# Fit the pipeline to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - all numeric, incl nans: ", accuracy)


Accuracy on sample data - all numeric, incl nans:  0.636


Now you know how to use preprocessing in pipelines with numeric data, and it looks like the accuracy has improved because of it! Text data preprocessing is next!

# 2. Text features and feature unions
## 2.1 Preprocessing text features
Here, you'll perform a similar preprocessing pipeline step, only this time you'll use the `text` column from the sample data.

To preprocess the text, you'll turn to `CountVectorizer()` to generate a bag-of-words representation of the data, as in Chapter 2. Using the [default](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) arguments, add a `(step, transform)` tuple to the steps list in your pipeline.

Make sure you select only the text column for splitting your training and test sets.

As usual, your `sample_df` is ready and waiting in the workspace.

### Instructions:
* Import `CountVectorizer` from `sklearn.feature_extraction.text`.
* Create training and test sets by selecting the correct subset of `sample_df`: `'text'`.
* Add the `CountVectorizer` step (with the name `'vec'`) to the correct position in the pipeline.
* Fit the pipeline to the training data and compute its accuracy.

In [4]:
# Import the CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Split out only the text data
X_train, X_test, y_train, y_test = train_test_split(sample_df['text'],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=456)

# Instantiate Pipeline object: pl
pl = Pipeline([
        ('vec', CountVectorizer()),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - just text data: ", accuracy)



Accuracy on sample data - just text data:  0.808


Looks like you're ready to create a pipeline for processing multiple datatypes!
## 2.2 Multiple types of processing: FunctionTransformer
The next two exercises will introduce new topics you'll need to make your pipeline truly excel.

Any step in the pipeline must be an object that implements the `fit` and `transform` methods. The [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) creates an object with these methods out of any Python function that you pass to it. We'll use it to help select subsets of data in a way that plays nicely with pipelines.

You are working with numeric data that needs imputation, and text data that needs to be converted into a bag-of-words. You'll create functions that separate the text from the numeric variables and see how the `.fit()` and `.transform()` methods work.

### Instructions:
* Compute the selector `get_text_data` by using a lambda function and `FunctionTransformer()` to obtain all `'text'` columns.
* Compute the selector `get_numeric_data` by using a lambda function and `FunctionTransformer()` to obtain all the numeric columns (including missing data). These are `'numeric'` and `'with_missing'`.
* Fit and transform `get_text_data` using the `.fit_transform()` method with `sample_df` as the argument.
* Fit and transform `get_numeric_data` using the same approach as above.

In [5]:
# Import FunctionTransformer
from sklearn.preprocessing import FunctionTransformer

# Obtain the text data: get_text_data
get_text_data = FunctionTransformer(lambda x: x['text'], validate=False)

# Obtain the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[['numeric', 'with_missing']], validate=False)

# Fit and transform the text data: just_text_data
just_text_data = get_text_data.fit_transform(sample_df)

# Fit and transform the numeric data: just_numeric_data
just_numeric_data = get_numeric_data.fit_transform(sample_df)

# Print head to check results
print('Text Data')
print(just_text_data.head())
print('\nNumeric Data')
print(just_numeric_data.head())

Text Data
0           
1        foo
2    foo bar
3           
4    foo bar
Name: text, dtype: object

Numeric Data
     numeric  with_missing
0 -10.856306      4.433240
1   9.973454      4.310229
2   2.829785      2.469828
3 -15.062947      2.852981
4  -5.786003      1.826475


You can see in the shell that fit and transform are now available to the selectors. Let's put the selectors to work!
## 2.3 Multiple types of processing: FeatureUnion
Now that you can separate text and numeric data in your pipeline, you're ready to perform separate steps on each by nesting pipelines and using `FeatureUnion()`.

These tools will allow you to streamline all preprocessing steps for your model, even when multiple datatypes are involved. Here, for example, you don't want to impute our text data, and you don't want to create a bag-of-words with our numeric data. Instead, you want to deal with these separately and then join the results together using `FeatureUnion()`.

In the end, you'll still have only two high-level steps in your pipeline: preprocessing and model instantiation. The difference is that the first preprocessing step actually consists of a pipeline for numeric data and a pipeline for text data. The results of those pipelines are joined using `FeatureUnion()`.

### Instructions:
* In the `process_and_join_features`:
    * Add the steps `('selector', get_numeric_data)` and `('imputer', Imputer())` to the `'numeric_features'` preprocessing step.
    * Add the equivalent steps for the `text_features` preprocessing step. That is, use `get_text_data` and a `CountVectorizer` step with the name `'vectorizer'`.
* Add the transform step `process_and_join_features` to `'union'` in the main pipeline, `pl`.

In [6]:
# Import FeatureUnion
from sklearn.pipeline import FeatureUnion

# Split using ALL data in sample_df
X_train, X_test, y_train, y_test = train_test_split(sample_df[['numeric', 'with_missing', 'text']],
                                                    pd.get_dummies(sample_df['label']), 
                                                    random_state=22)

# Create a FeatureUnion with nested pipeline: process_and_join_features
process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )

# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='liblinear')))
    ])


# Fit pl to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on sample data - all data: ", accuracy)



Accuracy on sample data - all data:  0.928
